In [24]:
import pandas as pd
import numpy as np
from telethon.sync import TelegramClient
from telethon.tl.types import PeerUser, PeerChat, PeerChannel, InputMessagesFilterPhotoVideo
from telethon.tl.functions.channels import GetFullChannelRequest
import json
import re
from telethon.errors import ChannelPrivateError
import pickle
import tqdm
import glob

In [2]:
with open('../credentials/credentials.pickle','rb') as file:
    api_id, api_hash, phone, username = pickle.load(file)

In [3]:
first_edgelist = pd.read_csv('../first_edgelist/first_edgelist.csv')

Let's start building a dictionnaire id_to_names and id_to_description.
We start from 'forward_to' of the first edgelist (there will be only Azione Incel and FDB)

In [4]:
with open('../dictionaires/id_to_names.json', 'r') as file:
    id_to_names = json.load(file)

with open('../dictionaires/id_to_desc.json', 'r') as file:
    id_to_desc = json.load(file)

In [11]:
id_to_names = dict(map(lambda x: (int(x[0]), x[1]),id_to_names.items())) #cast into integer
id_to_desc = dict(map(lambda x: (int(x[0]), x[1]),id_to_desc.items()))

In [12]:
unique_values = np.setdiff1d(first_edgelist['forward_from'], first_edgelist['forward_to'])
#get only values in "forward_from" that ARE NOT in "forward_to" because we already did every "forward_to"

In [13]:
unique_values

array([1175952757, 1177310741, 1182409216, 1216001239, 1216987129,
       1226866970, 1241907375, 1242031478, 1281032141, 1318655547,
       1355997151, 1392773702, 1455830529, 1480147271, 1506064089,
       1510627978, 1553618785, 1554578584, 1573599559, 1586195079,
       1611808700, 1623963447, 1636550716, 1655441290, 1659816199,
       1691299588, 1712348366, 1713781188, 1723813463, 1729608542,
       1751034477, 1757951604, 1768039844, 1787190842, 1796096784,
       1849977466, 1850972694, 1854612234, 1856892401, 1871322814,
       1876121761, 1879320718, 1879647616, 1919608063, 1956399677,
       1985819232, 2071879003, 2098891001, 2101278126, 2114197819,
       2138457338, 2139720080, 2142434931])

In [16]:
id_to_desc

{1344471586: 'Telegram https://t.me/CentroStvdiFascismo\nSito https://centrostvdifascismo.wordpress.com\nCinematografo https://odysee.com/@CentroStvdiFascismo\nWhatsapp https://whatsapp.com/channel/0029VaIejYSF1YlXO33jX32X\nContatti centrostudifascismo@virgilio.it',
 1394864973: "Canale ufficiale del movimento studentesco più rivoluzionario d'Italia - dal 2006"}

In [17]:
import datetime

In [18]:
offset_date = datetime.datetime(2023,10,1)

In [19]:
def get_channels_forward(lista_messaggi, receving_id):
    """
    lista_messaggi: list of patched telethon messages
    receving_id: id of the scraped group. So the forwarded_to id

    returns list of tuple with format (forwarded_from, forwarded_to)
    where forwarded_to is the receving_id

    """
    lista_fwd = []
    for mex in lista_messaggi:
        dict_mex = mex.to_dict()
        if 'fwd_from' in dict_mex.keys() and dict_mex['fwd_from'] is not None:
            fwd_info = dict_mex['fwd_from']['from_id']
            if type(fwd_info) == dict and fwd_info['_']=='PeerChannel':
                lista_fwd.append((fwd_info['channel_id'], receving_id))

    return lista_fwd
        

In [20]:
async def get_data_channel(channel_id, offset_date, names_dict, desc_dict):
    """
    Get all messages from channel NEWER than offset date (e.g. sent after offset date)

    args:

    channel_id: id of channel to be scraped (has to be an integer!)
    offset_date: date limit 
    names_dict: dict of id-to-names
    desc_dict: dict of id-to-bio
    
    return list of patcthed messages
    """
    
    async with TelegramClient(username, api_id, api_hash) as client:
                entity = await client.get_entity(PeerChannel(channel_id))
                full_entity = await client(GetFullChannelRequest(channel=entity))

                names_dict[channel_id] = entity.title #get group name
                desc_dict[channel_id] = full_entity.full_chat.about #get group bio

                lista_mex = await client.get_messages(entity, reverse=True, offset_date=offset_date, limit=None)
    
    return lista_mex

In [29]:
if glob.glob('../temporary_data/*.pickle'):
    with open('../temporary_data/temporary_third_edgelist.pickle','rb') as file_edgelist,\
        open('../temporary_data/temporary_id_to_names.pickle','rb') as file_id_names,\
        open('../temporary_data/temporary_id_to_desc.pickle','rb') as file_id_desc,\
        open('../temporary_data/temporary_index.pickle','rb') as index_file:

            third_edgelist_list = pickle.load(file_edgelist)
            id_to_names = pickle.load(file_id_names)
            id_to_desc = pickle.load(file_id_desc)
            index = pickle.load(index_file)

else:
    second_edgelist_list = []
    index=0

In [30]:
index

0

In [31]:
start_value = index
final_value = len(unique_values)
for index in range(start_value,final_value):
    codice = unique_values[index]
    try:
        list_messages = await get_data_channel(int(codice), offset_date, id_to_names, id_to_desc)
        list_tuples = get_channels_forward(list_messages, codice)
        second_edgelist_list.extend(list_tuples)
    except(ChannelPrivateError, ValueError) as e:
        print(e)

The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetChannelsRequest)
The channel specified is private and you lack permission to access it. Another reason may be that you were banned from it (caused by GetChannelsRequest)


In [35]:
forward_from = list(map(lambda x: x[0], second_edgelist_list))
forward_to = list(map(lambda x: x[1], second_edgelist_list))

In [36]:
second_edgelist_df_raw = pd.DataFrame({'forward_from':forward_from,\
    'forward_to':forward_to})

In [39]:
#save edgelist
second_edgelist = second_edgelist_df_raw.value_counts().reset_index()
second_edgelist.to_csv('../second_edgelist/second_edgelist.csv',index=False)

In [43]:
#need to cast keys as int (instead of int64) to dump json
id_to_names_int = dict(map(lambda x: (int(x[0]),x[1]),id_to_names.items()))
id_to_desc_int = dict(map(lambda x: (int(x[0]),x[1]),id_to_desc.items()))

In [44]:
with open('../dictionaires/id_to_names.json','w') as file_names, \
    open('../dictionaires/id_to_desc.json','w') as file_desc:
    json.dump(id_to_names_int, file_names)
    json.dump(id_to_desc_int, file_desc)